In [1]:
######################################################################################### 
# Read secretes from Azure Key Vault
#########################################################################################
## This is the name of my Azure Key Vault 
key_vault = "https://domain-keyvault.vault.azure.net/"
## I have stored my tenant id as one of the secrets to make it easier to use when needed 
tenant_id = mssparkutils.credentials.getSecret(key_vault , "tenantid") 
## This is my application Id for my service principal account 
client_id = mssparkutils.credentials.getSecret(key_vault , "pbi-applicationid") 
## This is my Client Secret for my service principal account 
client_secret = mssparkutils.credentials.getSecret(key_vault , "powerbi-clientsecret") 

StatementMeta(, dd839073-cc86-48a0-81f5-fcb19d878d33, 3, Finished, Available, Finished)

In [2]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
import msal
import requests
import json
import datetime
from datetime import datetime,date,timedelta

# Replace with your Azure AD app details
# client_id = "YOUR_CLIENT_ID"
# client_secret = "YOUR_CLIENT_SECRET"
# tenant_id = "YOUR_TENANT_ID"


authority_url = f"https://login.microsoftonline.com/{tenant_id}"
graph_api_url = "https://graph.microsoft.com/v1.0"

# Create MSAL client application
app = msal.ConfidentialClientApplication(
    client_id, authority=authority_url, client_credential=client_secret
)

# Function to get an access token
def get_access_token():
    token_response = app.acquire_token_for_client(scopes=["https://graph.microsoft.com/.default"])
    return token_response.get("access_token")

# Function to get all groups
def get_all_groups(access_token):
    groups_url = f"{graph_api_url}/groups"
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(groups_url, headers=headers)
    return response.json().get("value", [])

# Function to get members of a specific group
def get_group_members(group_id, access_token):
    members_url = f"{graph_api_url}/groups/{group_id}/members"
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(members_url, headers=headers)
    return response.json().get("value", [])

# Main logic to fetch groups and members
def main():
    access_token = get_access_token()
    
    if not access_token:
        print("Failed to obtain access token.")
        return

    all_groups_data = []

    groups = get_all_groups(access_token)

    for group in groups:
        group_id = group["id"]
        group_name = group["displayName"]
        print(f"Fetching members for group: {group_name}")
        
        members = get_group_members(group_id, access_token)
        member_list = [{"id": member["id"], "displayName": member.get("displayName", "N/A"),"jobTitle": member.get("jobTitle", "N/A"),"companyName": member.get("companyName", "N/A")} for member in members]
        
        group_data = {
            "group_id": group_id,
            "group_name": group_name,
            "members": member_list
        }
        
        all_groups_data.append(group_data)

    
    # Create Directory if it does not exist
    mssparkutils.fs.mkdirs("Files/Entra_Id/")

    fileName = 'groups_and_members_' + (datetime.today()).strftime('%Y%m%d') + '.json'

    # Write the output to a JSON file
    with open(f"/lakehouse/default/Files/Entra_Id/{fileName}", "w") as json_file:
        json.dump(all_groups_data, json_file, indent=4)
    # with open('/lakehouse/default/Files/Entra_Id/groups_and_members_' + (datetime.today()).strftime('%Y%m%d') + '.json', 'w') as file:
    #     json_data = json.dumps(all_groups_data)
        # file.write(json_data.encode('utf-8'))
    print("Data written to groups_and_members.json")

if __name__ == "__main__":
    main()


StatementMeta(, dd839073-cc86-48a0-81f5-fcb19d878d33, 4, Finished, Available, Finished)

Data written to groups_and_members.json
